In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
pi = np.pi

# obrazek powinien byc tak
# /\
# |
# |
# |
# (0,0)------->

In [ ]:
img = mpimg.imread('tomograf-zdjecia/Kwadraty2.jpg')[:,:,2] / 255

picHeight = img.shape[0]
picWidth = img.shape[1]

r =  np.sqrt( picWidth**2 + picHeight**2 ) /2

print(picWidth)
print(picWidth)
print(r)

#plt.imshow(img)

In [ ]:
def transformToPictureCoordinates(point, picHeight, picWidth):
    X = point[0] + (0.5*picWidth)
    Y = point[1] + (0.5*picHeight)
    
    X = int(np.floor(X))
    Y = int(np.floor(Y))
    return ( X, Y )

def getSource(alfa, r):
    Xe = r * np.cos(alfa)
    Ye = r * np.sin(alfa)
    
    return (Xe, Ye)

#n - ilosc detektorow, alfa - przesuniecie wszystkeigo, phi - rozstawienie detektorow, r - promien kola
def getDetectors(n, alfa, phi, r):
    detectors = []
    
    # D0
    Xd = r * np.cos(alfa + pi - phi/2)    
    Yd = r * np.sin(alfa + pi - phi/2)

    detectors.append( (Xd, Yd) )
    
    # D1 ... Dn-1
    for i in range(1,n):
        Xd = r * np.cos(alfa + pi - phi/2 + i * phi/(n-1))
        Yd = r * np.sin(alfa + pi - phi/2 + i * phi/(n-1))
        
        detectors.append( (Xd, Yd) )
    
    return detectors

In [ ]:
def bresenhamL(x1, y1, x2, y2):
    points = []
    dx = x2 - x1
    dy = y2 - y1
    m = 1
    if dy < 0:
        m = -1
        dy = -dy
    j = y1
    e = (2 * dy) - dx
    
    for i in range(x1, x2):
        points.append((i, j))
        if e > 0:
            j += m
            e += 2 * (dy - dx)
        else:
            e += 2 * dy
    return points

def bresenhamH(x1, y1, x2, y2):
    points = []
    dx = x2 - x1
    dy = y2 - y1
    m = 1
    if dx < 0:
        m = -1
        dx = -dx
    i = x1
    e = (2 * dx) - dy
    
    for j in range(y1, y2):
        points.append((i, j))
        if e > 0:
            i += m
            e += 2 * (dx - dy)
        else:
            e += 2 * dx
    return points

def bresenham(x1, y1, x2, y2):
    if abs(y2 - y1) < abs(x2 - x1):
        if x1 > x2:
            return bresenhamL(x2, y2, x1, y1)
        else:
            return bresenhamL(x1, y1, x2, y2)
    else:
        if y1 > y2:
            return bresenhamH(x2, y2, x1, y1)
        else:
            return bresenhamH(x1, y1, x2, y2)

In [ ]:
def absorption(img, x1, y1, x2, y2):
    height = img.shape[0]
    width = img.shape[1]
    
    s = 0
    
    for (x, y) in bresenham(x1, y1, x2, y2):
        if x >= 0 and x < width and y >= 0 and y < height:
            s += img[x][y]
    return s

In [ ]:
def img2sinogram(img, alfaDelta, n, phi):
    alfaDelta = np.radians(alfaDelta)
    phi = np.radians(phi)
    
    picHeight = img.shape[0]
    picWidth = img.shape[1]
    d = np.sqrt( picWidth**2 + picHeight**2 )
    r =  d / 2
    
    lines = []
    alfa = 0
    while alfa < 2 * pi:
        
        source = getSource(alfa, r)
        pSource = transformToPictureCoordinates(source, picWidth, picHeight)
        
        detectors = getDetectors(n, alfa, phi, r)
        pDetectors = [transformToPictureCoordinates(x, picWidth, picHeight) for x in detectors]
        
        line = []
        
        for detector in pDetectors:
            a = absorption(img, pSource[0], pSource[1], detector[0], detector[1])
            line.append(a)
            
        
        lines.append(line)
        alfa += alfaDelta
    
    plt.imshow(lines, cmap='binary_r')
        


In [ ]:
img2sinogram(img, 2, 180, 180)

In [ ]:
# TESTY

# wspolrzedne abstrakcyjne
source = getSource(pi+pi/4, r)
detectors = getDetectors(3, pi+pi/4, pi/2, r)

# wspolrzedne na obrazku
pSource = transformToPictureCoordinates(source, picWidth, picHeight)
pDetectors = [transformToPictureCoordinates(x, picWidth, picHeight) for x in detectors]

print(pSource)
print(pDetectors)
# ------------------